<a href="https://colab.research.google.com/github/JuelGeorge/Adaptive_BCI_2D/blob/main/ErrpGanTrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow tensorflow-gpu matplotlib tensorflow-datasets ipywidgets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
import tensorflow_datasets as tfds
from matplotlib import pyplot as plt

In [ ]:
# Bring in the sequential api for the generator and discriminator
from tensorflow.keras.models import Sequential
# Bring in the layers for the neural network
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape, LeakyReLU, Dropout, UpSampling2D, UpSampling1D, Conv1D,Resizing, BatchNormalization


In [ ]:
def build_discriminator(): 
    model = Sequential()
    
    # First Conv Block
    model.add(UpSampling2D(size=(1,2)))
    model.add(Conv2D(64, 4, input_shape = (1,128,64)))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    #model.add(BatchNormalization())
    '''
    
    # Second Conv Block
    model.add(UpSampling2D(size=(1,2)))
    model.add(Conv2D(64, 8))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    
    
    # Third Conv Block
    model.add(UpSampling2D(size=(1,2))) 
    model.add(Conv2D(64, 16))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    
    # Fourth Conv Block
    model.add(UpSampling2D(size=(1,2)))
    model.add(Conv2D(64, 32))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())

     # Fourth Conv Block
    model.add(UpSampling2D(size=(1,2)))
    model.add(Conv2D(64, 64))
    model.add(LeakyReLU(0.2))
    model.add(Dropout(0.4))
    model.add(BatchNormalization())
    
    # Flatten then pass to dense layer
    model.add(Flatten())
    model.add(Dropout(0.4))
    model.add(Dense(1, activation='sigmoid'))

    model.add(Dense(103,activation='relu'))

    model.add(Dense(2,activation='softmax'))
    '''
    
    return model 

In [ ]:
discriminator = build_discriminator()
discriminator.summary()

ValueError: ignored

In [1]:
# Adam is going to be the optimizer for both
from tensorflow.keras.optimizers import Adam
# Binary cross entropy is going to be the loss for both 
from tensorflow.keras.losses import BinaryCrossentropy

In [2]:
g_opt = Adam(learning_rate=0.0001) 
d_opt = Adam(learning_rate=0.00001) 
g_loss = BinaryCrossentropy()
d_loss = BinaryCrossentropy()

In [5]:
from keras.models import Model

In [6]:
class FashionGAN(Model): 
    def __init__(self, generator, discriminator, *args, **kwargs):
        # Pass through args and kwargs to base class 
        super().__init__(*args, **kwargs)
        
        # Create attributes for gen and disc
        self.generator = generator 
        self.discriminator = discriminator 
        
    def compile(self, g_opt, d_opt, g_loss, d_loss, *args, **kwargs): 
        # Compile with base class
        super().compile(*args, **kwargs)
        
        # Create attributes for losses and optimizers
        self.g_opt = g_opt
        self.d_opt = d_opt
        self.g_loss = g_loss
        self.d_loss = d_loss 

    def train_step(self, batch):
        # Get the data 
        real_images = batch
        fake_images = self.generator(tf.random.normal((1,128,64)), training=False)
        
        # Train the discriminator
        with tf.GradientTape() as d_tape: 
            # Pass the real and fake images to the discriminator model
            yhat_real = self.discriminator(real_images, training=True) 
            yhat_fake = self.discriminator(fake_images, training=True)
            yhat_realfake = tf.concat([yhat_real, yhat_fake], axis=0)
            
            # Create labels for real and fakes images
            y_realfake = tf.concat([tf.zeros_like(yhat_real), tf.ones_like(yhat_fake)], axis=0)
            
            # Add some noise to the TRUE outputs
            noise_real = 0.15*tf.random.uniform(tf.shape(yhat_real))
            noise_fake = -0.15*tf.random.uniform(tf.shape(yhat_fake))
            y_realfake += tf.concat([noise_real, noise_fake], axis=0)
            
            # Calculate loss - BINARYCROSS 
            total_d_loss = self.d_loss(y_realfake, yhat_realfake)
            
        # Apply backpropagation - nn learn 
        dgrad = d_tape.gradient(total_d_loss, self.discriminator.trainable_variables) 
        self.d_opt.apply_gradients(zip(dgrad, self.discriminator.trainable_variables))
         # Train the generator 
        with tf.GradientTape() as g_tape: 
            # Generate some new images
            gen_images = self.generator(tf.random.normal((1,128,64)), training=True)
                                        
            # Create the predicted labels
            predicted_labels = self.discriminator(gen_images, training=False)
                                        
            # Calculate loss - trick to training to fake out the discriminator
            total_g_loss = self.g_loss(tf.zeros_like(predicted_labels), predicted_labels) 
            
        # Apply backprop
        ggrad = g_tape.gradient(total_g_loss, self.generator.trainable_variables)
        self.g_opt.apply_gradients(zip(ggrad, self.generator.trainable_variables))
        
        return {"d_loss":total_d_loss, "g_loss":total_g_loss}

In [9]:
fashgan = FashionGAN(generator, discriminator)

NameError: ignored

In [8]:
fashgan.compile(g_opt, d_opt, g_loss, d_loss)

NameError: ignored